In [37]:
import re
import string
from pickle import dump
from unicodedata import normalize
from numpy import array
from keras.preprocessing.text import Tokenizer
import os
import numpy as np
os.environ["CUDA_VISIBLE_DEVICES"]="7"
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, mode='rt', encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [38]:
# split a loaded document into sentences
def to_pairs(doc):
	lines = doc.strip().split('\n')
	pairs = [line.split('\t') for line in  lines]
	return pairs

In [39]:
# clean a list of lines
def clean_pairs(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
            line = [word.translate(table) for word in line]
            # remove non-printable chars form each token
            line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return array(cleaned)

In [40]:
# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)

# # load dataset
# filename = 'parallel/deu.txt'
# doc = load_doc(filename)
# # split into english-german pairs
# pairs = to_pairs(doc)
# # clean sentences
# cp = clean_pairs(pairs)
# # save clean pairs to file
# save_clean_data(cp, 'old.pkl')
# # spot check

filename = 'sub.en-hi.en'
eng = load_doc(filename).strip().split('\n')

filename = 'sub.en-hi.hi'
hin = load_doc(filename).strip().split('\n')


pairs = []

# for i in range(len(eng)):
#     pairs.append([eng[i], hin[i]])
# split into english-german pairs




In [41]:
# line = pairs[500][1]
# line = line.split()
re_print = re.compile('[^%s]' % re.escape(string.printable))
table = str.maketrans('', '', string.punctuation)
eng2 = []
for line in eng:
    line = normalize('NFD', line).encode('ascii', 'ignore')
    line = line.decode('UTF-8')
    # tokenize on white space
    line = line.split()
    # convert to lowercase
    line = [word.lower() for word in line]
    # remove punctuation from each token
    line = [word.translate(table) for word in line]
    # remove non-printable chars form each token
    line = [re_print.sub('', w) for w in line]
    # remove tokens with numbers in them
    line = [word for word in line if word.isalpha()]
    eng2.append(' '.join(line))


In [42]:
pairs = list()
for i in range(len(eng2)):
    pair = list()
    pair.append(eng2[i])
    pair.append(hin[i])
    pairs.append(pair)


In [43]:
def get_only_small(pairs, MAX_LEN=10, MAX_LEN2=20):
    new_pairs = list()
    for pair in pairs:
        line = pair[0] #english line maxlength
        if(len(line.strip().split(' ')) <= MAX_LEN and len(pair[1].strip().split(' ')) <= MAX_LEN2):
            pair2 = list()
            pair2.append(pair[0])
            pair2.append(pair[1])
            new_pairs.append(pair2)
    return new_pairs

In [44]:
cp = get_only_small(pairs, 10, 10)

In [46]:
# len(cp)
cp = array(cp)
save_clean_data(cp, 'hindi.pkl')

77360

In [48]:
from pickle import load
from pickle import dump
from numpy.random import rand
from numpy.random import shuffle

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)

# load dataset
raw_dataset = load_clean_sentences('hindi.pkl')
shuffle(raw_dataset)
# reduce dataset size
n_sentences = 77360
n_train = int(77360*0.9)
dataset = raw_dataset[:n_sentences]
# random shuffle

# split into train/test
train, test = dataset[:n_train], dataset[n_train:]
# save
save_clean_data(dataset, 'hindi-both.pkl')
save_clean_data(train, 'hindi-train.pkl')
save_clean_data(test, 'hindi-test.pkl')

Saved: hindi-both.pkl
Saved: hindi-train.pkl
Saved: hindi-test.pkl


In [49]:
# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# load datasets
dataset = load_clean_sentences('hindi-both.pkl')
train = load_clean_sentences('hindi-train.pkl')
test = load_clean_sentences('hindi-test.pkl')

# dataset = load_clean_sentences('english-german-10-10-both.pkl')
# train = load_clean_sentences('english-german-10-10-train.pkl')
# test = load_clean_sentences('english-german-10-10-test.pkl')

In [34]:

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer(num_words=1000)
	tokenizer.fit_on_texts(lines)
	return tokenizer


In [33]:
dataset[1000]

array(['hello vincent', '- हैलो विन्सेंट।'], dtype='<U91')

In [52]:
# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

In [35]:
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))
# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
print('German Vocabulary Size: %d' % ger_vocab_size)
print('German Max Length: %d' % (ger_length))

English Vocabulary Size: 15567
English Max Length: 10
German Vocabulary Size: 21617
German Max Length: 10


In [36]:
len(eng_tokenizer.word_index)

15566

In [55]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


In [1]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
# from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint
import os
os.environ["CUDA_VISIBLE_DEVICES"]="7"
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)


# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer(num_words=10000)
	tokenizer.fit_on_texts(lines)
	return tokenizer

# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

# one hot encode target sequence
def encode_output(sequences, vocab_size):
	ylist = list()
	for sequence in sequences:
		encoded = to_categorical(sequence, num_classes=vocab_size)
		ylist.append(encoded)
	y = array(ylist)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y


# load datasets
dataset = load_clean_sentences('hindi-both.pkl')
train = load_clean_sentences('hindi-train.pkl')
test = load_clean_sentences('hindi-test.pkl')

# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))
# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
print('German Vocabulary Size: %d' % ger_vocab_size)
print('German Max Length: %d' % (ger_length))

eng_length = ger_length
print('Set eng_length to ger_length')
# prepare training data
print('Encoding trainX')
trainX = encode_sequences(ger_tokenizer, ger_length, train[:, 1])
trainY = encode_sequences(eng_tokenizer, eng_length, train[:, 0])
print('Encoding trainY...')
trainY = encode_output(trainY, eng_vocab_size)
print('Done')
# prepare validation data
testX = encode_sequences(ger_tokenizer, ger_length, test[:, 1])
testY = encode_sequences(eng_tokenizer, eng_length, test[:, 0])
print('Encoding testY...')
testY = encode_output(testY, eng_vocab_size)
print('Done')


Using TensorFlow backend.


English Vocabulary Size: 15567
English Max Length: 10
German Vocabulary Size: 21617
German Max Length: 10
Set eng_length to ger_length
Encoding trainX
Encoding trainY...
Done
Encoding testY...
Done


In [2]:
i = 0
for key, value in ger_tokenizer.word_index.items():
    i += 1
    print(key)
    if i>1:
        break

कानूनों
सोना।


In [ ]:
# define NMT model
from keras.layers import RNN
from keras.layers import Bidirectional
from custom_recurrents import AttentionDecoder
from keras_self_attention import SeqSelfAttention

# def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
#     model = Sequential()
#     model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
# #     model.add(SeqSelfAttention(attention_activation='sigmoid'))
#     model.add(LSTM(n_units, return_sequences=False))
    
# #     model.add(AttentionDecoder(256, 256))
#     model.add(RepeatVector(tar_timesteps))
#     model.add(LSTM(n_units, return_sequences=True))
#     model.add(SeqSelfAttention(attention_activation='sigmoid'))
#     model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
#     return model


def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
    model = Sequential()
    model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
    model.add(Bidirectional(LSTM(n_units, return_sequences=True)))
    model.add(AttentionDecoder(128, tar_vocab, return_probabilities=True))
#     model.add(RepeatVector(tar_timesteps))
#     model.add(LSTM(n_units, return_sequences=True))
#     model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
    return model


# define model
model = define_model(ger_vocab_size, eng_vocab_size, ger_length, eng_length, 128)
model.compile(optimizer='adam', loss='categorical_crossentropy')
# summarize defined model
print(model.summary())
# plot_model(model, to_file='model.png', show_shapes=True)
# fit model
filename = 'model_30march_hindi_attention.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')


In [14]:
model.fit(trainX, trainY, epochs=5, batch_size=32, validation_data=(testX[:1000], testY[:1000]), callbacks=[checkpoint], verbose=1)

Train on 69624 samples, validate on 1000 samples
Epoch 1/5
69624/69624 [==============================] - 313s 4ms/step - loss: 1.9472 - val_loss: 2.1770
Epoch 2/5
69624/69624 [==============================] - 165s 2ms/step - loss: 1.8141 - val_loss: 2.1336
Epoch 3/5
69624/69624 [==============================] - 328s 5ms/step - loss: 1.6929 - val_loss: 2.0931
Epoch 4/5
69624/69624 [==============================] - 305s 4ms/step - loss: 1.5845 - val_loss: 2.0662
Epoch 5/5
69624/69624 [==============================] - 258s 4ms/step - loss: 1.4905 - val_loss: 2.0496


In [20]:
len(trainX[0])

17

In [9]:
from keras.models import load_model
source = testX[0]
from numpy import argmax

# load model
# model = load_model('model_27march_mymodel2.h5')
# generate target given source sequence
def predict_sequence(model, tokenizer, source):
	source = source.reshape((1, source.shape[0]))
	prediction = model.predict(source, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)

# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None


In [15]:
for i in range(len(test)-20, len(test)):
    print(test[i][0],'|', test[i][1], '|', predict_sequence(model, eng_tokenizer, testX[i]))

bullshit | - बकवास! | bullshit
imagine | कल्पना करो! | lmagine
your dad used to say | तुम्हारे पिताजी कहा करते थे: | you you you you
distract the alpha | ऐल्फ़ा का ध्यान बँटाओ। | the the the the the
he found his son | -उसे उसका बेटा मिल गया! | he her her her
no perfection anymore | अब और नहीं पूर्णता. | and and and the
im right behind you | मैं ठीक तुम्हारे पीछे ही हूँ! | im you you
my enemies are in the red keep | मेरे दुश्मन लाल रखें में हैं। | my my my my my
and they harvest what they need | और वे वे क्या जरूरत फसल. | they they they they they
he trusts you | उन्होंने कहा कि आप पर भरोसा करता है। | he he believe
automated female voice | स्वचालित महिला की आवाज: | voice voice voice
heres your food | यहाँ अपने भोजन है. | your your
be my guest | मेरे मेहमान हो। | my my my
and would i be sure that this | ♪ और अगर मुझे यकीन होता कि यह ♪ | if if if if if if again
im sorry hes not here | मुझे खेद है, वह यहाँ नहीं हूँ. | im not not not
im real sorry about your loss | मैं अपने नुकसान के बारे मे

In [49]:
text = "Er sagte, es sei beabsichtigt, eine Kontamination zu vermeiden"
mine = encode_sequences(ger_tokenizer, ger_length, [text])
predict_sequence(model, eng_tokenizer, mine[0])

'he said it was to a haircut'

In [58]:
import numpy as np
np.save('ger_tokenizer', ger_tokenizer)
np.save('eng_tokenizer', eng_tokenizer)

array([ 22,   3, 105, 588,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0], dtype=int32)

In [50]:
testX[0]

array([   8,  196,   16,  305,   25,    1,  855,   34,  100, 5006,   15,
          0,    0,    0,    0,    0,    0], dtype=int32)

In [12]:
# evaluate the skill of the model
from nltk.translate.bleu_score import corpus_bleu
def evaluate_model(model, tokenizer, sources, raw_dataset):
    actual, predicted = list(), list()
    for i, source in enumerate(sources):
        # translate encoded source text
#       source = source.reshape((1, source.shape[0]))
        translation = predict_sequence(model, eng_tokenizer, source)
        raw_target, raw_src = raw_dataset[i]
#       if i < 10:
#           print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
        actual.append(raw_target.split())
        predicted.append(translation.split())
        if i>1000:
            break
#   actual = [[['this', 'is', 'a', 'test']], [['this', 'is', 'a', 'test']]]
#   predicted = [['this', 'is', 'a', 'test'], ['this', 'is', 'a', 'test']]
    actual2 = []
    for a in actual:
        actual2.append([a])
    actual = actual2

    # calculate BLEU score
    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

/home/apoorv/.local/lib/python3.5/site-packages/sklearn/externals/joblib/_multiprocessing_helpers.py:38: UserWarning: [Errno 12] Cannot allocate memory.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [16]:
evaluate_model(model, eng_tokenizer, testX, test)

BLEU-1: 0.275918
BLEU-2: 0.115854
BLEU-3: 0.062558
BLEU-4: 0.015397


In [98]:
# 1-gram individual BLEU
from nltk.translate.bleu_score import sentence_bleu
reference = [['meet', 'me', 'in', 'an', 'hour']]
candidate = ['me', 'in' ,'an' ,'hour']
score = sentence_bleu(reference, candidate,weights=(0.25, 0.25,0.25, 0.25))
print(score)

0.7788007830714049
